<a href="https://colab.research.google.com/github/galuf/BC-prot/blob/master/CamposClasification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install patool

     |████████████████████████████████| 81kB 3.8MB/s 


In [2]:
import numpy as np
import os
import re
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import keras
from keras.utils import to_categorical
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

import patoolib

In [3]:
from google.colab import drive
drive.mount('/content/drive')

patoolib.extract_archive("drive/My Drive/CC022-UNI/Lab4/sportimages.zip", outdir="sample_data")
print(os.getcwd())
!ls sample_data/sportimages

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
patool: Extracting drive/My Drive/CC022-UNI/Lab4/sportimages.zip ...
patool: running /usr/bin/7z x -osample_data -- "drive/My Drive/CC022-UNI/Lab4/sportimages.zip"
patool: ... drive/My Drive/CC022-UNI/Lab4/sportimages.zip extracted to `sample_data'.
/content
americano  basket  beisball  boxeo  ciclismo  f1  futbol  golf	natacion  tenis


In [4]:
dirname = os.path.join(os.getcwd(), 'sample_data/sportimages')
imgpath = dirname + os.sep 
 
images = []
directories = []
dircount = []
prevRoot=''
cant=0
 
print("leyendo imagenes de ",imgpath)
 
for root, dirnames, filenames in os.walk(imgpath):
    for filename in filenames:
        if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
            cant=cant+1
            filepath = os.path.join(root, filename)
            image = plt.imread(filepath)
            images.append(image)
            b = "Leyendo..." + str(cant)
            print (b, end="\r")
            if prevRoot !=root:
                print(root, cant)
                prevRoot=root
                directories.append(root)
                dircount.append(cant)
                cant=0
dircount.append(cant)
 
dircount = dircount[1:]
dircount[0]=dircount[0]+1
print('Directorios leidos:',len(directories))
print("Imagenes en cada directorio", dircount)
print('suma Total de imagenes en subdirs:',sum(dircount))

leyendo imagenes de  /content/sample_data/sportimages/
/content/sample_data/sportimages/golf 1
/content/sample_data/sportimages/f1 9768
/content/sample_data/sportimages/basket 5053
/content/sample_data/sportimages/tenis 8823
/content/sample_data/sportimages/futbol 8937
/content/sample_data/sportimages/boxeo 7617
/content/sample_data/sportimages/natacion 7125
/content/sample_data/sportimages/americano 5172
/content/sample_data/sportimages/beisball 9348
/content/sample_data/sportimages/ciclismo 7752
Directorios leidos: 10
Imagenes en cada directorio [9769, 5053, 8823, 8937, 7617, 7125, 5172, 9348, 7752, 7532]
suma Total de imagenes en subdirs: 77128


In [5]:
labels=[]
indice=0
for cantidad in dircount:
    for i in range(cantidad):
        labels.append(indice)
    indice=indice+1
print("Cantidad etiquetas creadas: ",len(labels))
 
deportes=[]
indice=0
for directorio in directories:
    name = directorio.split(os.sep)
    print(indice , name[len(name)-1])
    deportes.append(name[len(name)-1])
    indice=indice+1
 
y = np.array(labels)
X = np.array(images, dtype=np.uint8) #convierto de lista a numpy
 
# Find the unique numbers from the train labels
classes = np.unique(y)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

Cantidad etiquetas creadas:  77128
0 golf
1 f1
2 basket
3 tenis
4 futbol
5 boxeo
6 natacion
7 americano
8 beisball
9 ciclismo
Total number of outputs :  10
Output classes :  [0 1 2 3 4 5 6 7 8 9]


In [6]:
#Mezclar todo y crear los grupos de entrenamiento y testing
train_X,test_X,train_Y,test_Y = train_test_split(X,y,test_size=0.2)
print('Training data shape : ', train_X.shape, train_Y.shape)
print('Testing data shape : ', test_X.shape, test_Y.shape)
 
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X = train_X / 255.
test_X = test_X / 255.
 
# Change the labels from categorical to one-hot encoding
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)
 
# Display the change for category label using one-hot encoding
print('Original label:', train_Y[0])
print('After conversion to one-hot:', train_Y_one_hot[0])
 
train_X,valid_X,train_label,valid_label = train_test_split(train_X, train_Y_one_hot, test_size=0.2, random_state=13)
 
print(train_X.shape,valid_X.shape,train_label.shape,valid_label.shape)

Training data shape :  (61702, 21, 28, 3) (61702,)
Testing data shape :  (15426, 21, 28, 3) (15426,)
Original label: 9
After conversion to one-hot: [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
(49361, 21, 28, 3) (12341, 21, 28, 3) (49361, 10) (12341, 10)


In [7]:
INIT_LR = 1e-3
epochs = 100
batch_size = 64
 
sport_model = Sequential()
sport_model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',padding='same',input_shape=(21,28,3)))
sport_model.add(LeakyReLU(alpha=0.1))
sport_model.add(MaxPooling2D((2, 2),padding='same'))
sport_model.add(Dropout(0.5))
 
sport_model.add(Flatten())
sport_model.add(Dense(32, activation='linear'))
sport_model.add(LeakyReLU(alpha=0.1))
sport_model.add(Dropout(0.5)) 
sport_model.add(Dense(nClasses, activation='softmax'))
 
sport_model.summary()
 
sport_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adagrad(lr=INIT_LR, decay=INIT_LR / 100),metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 21, 28, 32)        896       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 21, 28, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 11, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 11, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4928)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                157728    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32)                0

In [8]:
sport_train_dropout = sport_model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))

# guardamos la red, para reutilizarla en el futuro, sin tener que volver a entrenar
sport_model.save("sports_mnist.h5py")

Epoch 1/500
772/772 [==============================] - 3s 4ms/step - loss: 2.1467 - accuracy: 0.2231 - val_loss: 1.8577 - val_accuracy: 0.4849
Epoch 2/500
772/772 [==============================] - 3s 4ms/step - loss: 1.8276 - accuracy: 0.3628 - val_loss: 1.5451 - val_accuracy: 0.6015
Epoch 3/500
772/772 [==============================] - 3s 4ms/step - loss: 1.6200 - accuracy: 0.4430 - val_loss: 1.3490 - val_accuracy: 0.6573
Epoch 4/500
772/772 [==============================] - 3s 4ms/step - loss: 1.4825 - accuracy: 0.4981 - val_loss: 1.2231 - val_accuracy: 0.6818
Epoch 5/500
772/772 [==============================] - 3s 4ms/step - loss: 1.3832 - accuracy: 0.5361 - val_loss: 1.1264 - val_accuracy: 0.7112
Epoch 6/500
772/772 [==============================] - 3s 4ms/step - loss: 1.3148 - accuracy: 0.5616 - val_loss: 1.0543 - val_accuracy: 0.7275
Epoch 7/500
772/772 [==============================] - 3s 4ms/step - loss: 1.2537 - accuracy: 0.5826 - val_loss: 0.9956 - val_accuracy: 0.7378

In [9]:
test_eval = sport_model.evaluate(test_X, test_Y_one_hot, verbose=1)
 
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

483/483 [==============================] - 1s 2ms/step - loss: 0.2184 - accuracy: 0.9448
Test loss: 0.2183580845594406
Test accuracy: 0.9448333978652954


In [467]:
#prediccion
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('drive/My Drive/CC022-UNI/Lab4/img/voley2.jpg', target_size=(21,28))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = sport_model.predict(test_image)
result

array([[0.0000000e+00, 1.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        1.1542511e-15, 0.0000000e+00]], dtype=float32)

In [468]:
if result[0][0] ==1:
    prediction = 'golf'
elif result[0][1] ==1.0:
    prediction = 'f1'
elif result[0][2] ==1:
    prediction = 'basket'
elif result[0][3] ==1:
    prediction = 'tenis'
elif result[0][4] ==1:
    prediction = 'futbol'
elif result[0][5] ==1:
    prediction = 'boxeo'
elif result[0][6] ==1:
    prediction = 'natacion'
elif result[0][7] ==1:
    prediction = 'americano'
elif result[0][8] ==1:
    prediction = 'beisball'
else :
    prediction = 'ciclismo'
print(prediction)

f1


In [470]:
test = keras.models.load_model("sports_mnist.h5py")

In [471]:
test_eval = test.evaluate(test_X, test_Y_one_hot, verbose=1)
 
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

483/483 [==============================] - 1s 2ms/step - loss: 0.2184 - accuracy: 0.9448
Test loss: 0.2183580845594406
Test accuracy: 0.9448333978652954
